In [2]:
import pandas as pd
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.patches import Rectangle

import locale
# locale.setlocale(locale.LC_ALL)

# %matplotlib widget

In [2]:
height = 19.42*0.393701
width  = 13*0.393701

# plt.style.use('seaborn-v0_8-whitegrid')
plt.style.use('default')

# size = 9
plt.rcParams['font.family'] =  'serif'
plt.rcParams['font.size'] = 8
# plt.rcParams['axes.titlesize'] = size
# plt.rcParams['axes.labelsize'] = size
# plt.rcParams['xtick.labelsize'] = size
# plt.rcParams['ytick.labelsize'] = size
# plt.rcParams['legend.fontsize'] = size
plt.rcParams['figure.figsize'] = (width, height) #(6.29, 6/10*6.29) <-- Figsize zum einbinden in ein Word-Dokument
plt.rcParams['lines.linewidth'] = 0.8

plt.rcParams["axes.xmargin"] = 0
# plt.rcParams["axes.ymargin"] = 0
plt.rcParams['legend.frameon'] = 'False'

In [4]:
calc_self_sufficiency = lambda total_consumption, energy_import: (total_consumption - energy_import) / total_consumption * 100
calc_self_consumption = lambda total_production, energy_export: (total_production - energy_export) / total_production * 100

def self_consumption_for_rolling(slice, df_in):
    df = df_in.loc[slice.index, :]
    # print(df)
    total_production = df['PV'].sum(axis=0)
    energy_export = df.loc[df['Grid'] > 0, 'Grid'].sum(axis=0)
    self_cons = calc_self_consumption(total_production, energy_export)
    return self_cons

def self_sufficiency_for_rolling(slice, df_in):
    df = df_in.loc[slice.index, :]
    # print(df)
    total_consumption = df['Load'].sum(axis=0)
    energy_import = df.loc[df['Grid'] < 0, 'Grid'].sum(axis=0)
    self_suff = calc_self_sufficiency(total_consumption, energy_import)
    return self_suff

In [12]:
# output_dir = Path('../../data/output/scenario_first_test/')
# output_dirs = Path('../../data/output/scenarios5').iterdir() #  optimized with self sufficiency but error in PV perfect prediction!!!
output_dirs = Path('../../data/output/scenarios6_ss').iterdir() #  optimized with self sufficiency
# scenarios4 issues: - horizon of the lstm predictor is not limited to 2 months, this changes prediction accuracy over the year
#                    - weird discharges at high pv production: possible error objective function

outputs = {}
outputs_table    = {}
self_consumption = {}
self_sufficiency = {}
rmse_bes         = {}
mae_bes          = {}

sc_no_flex       = {}
ss_no_flex       = {}


slice2010 = slice(pd.to_datetime('2010-01-01 01:00:00+01:00'), pd.to_datetime('2010-12-31 23:00:00+01:00'))

for output_dir in output_dirs:
    output_grid          = pd.read_pickle(output_dir.joinpath('Grid_.pkl'))[slice2010]
    output_load          = pd.read_pickle(output_dir.joinpath('LoadModel_.pkl'))[slice2010]
    output_pv            = pd.read_pickle(output_dir.joinpath('PVModel_.pkl'))[slice2010]
    output_bes           = pd.read_pickle(output_dir.joinpath('StorageModel_.pkl'))[slice2010]
    output_pod           = pd.read_pickle(output_dir.joinpath('POD_.pkl'))[slice2010]
    # output_mpc           = pd.read_pickle(output_dir.joinpath('MPController_.pkl'))
    # output_dso_ec        = pd.read_pickle(output_dir.joinpath('DSOModel_0_electricity_ec.pkl'))
    # output_dso_esup      = pd.read_pickle(output_dir.joinpath('DSOModel_0_electricity_supplier.pkl'))
    # output_dso_nodedata  = pd.read_pickle(output_dir.joinpath('DSOModel_0_node_data.pkl'))

    # df = pd.concat([output_grid, output_load, output_pv, output_bes, output_pod], axis=1, keys=['grid', 'load', 'pv', 'bes', 'pod'], names=('simulator', 'models', 'attribute'))  # , 'mpc', output_mpc

    pv_power    = output_pv.xs('pv_power', axis=1, level='attribute').sum(axis=1).rename('PV')
    load        = output_load.xs('load', axis=1, level='attribute').sum(axis=1).rename('Load')
    bes         = output_bes[('StorageModel_0', 'power')].rename('BES')
    grid_power  = output_grid[('grid_0', 'residual power')].rename('Grid')

    df = pd.concat([pv_power, load, bes, grid_power], axis=1)  # , output_mpc

    df_out = pd.DataFrame([])
    df_out['self_sufficiency_rolling'] = df.rolling(pd.Timedelta(days=30))['Load'].apply(self_sufficiency_for_rolling, args=(df, ))  # apply(self_sufficiency_for_rolling)  # pd.Timedelta(hours=4)
    df_out['self_consumption_rolling'] = df.rolling(pd.Timedelta(days=30))['Load'].apply(self_consumption_for_rolling, args=(df, ))  # apply(self_sufficiency_for_rolling)  # pd.Timedelta(hours=4)

    df_out['self_sufficiency_monthly'] = df.groupby(df.index.month)['Load'].apply(self_sufficiency_for_rolling, df)  # apply(self_sufficiency_for_rolling)  # pd.Timedelta(hours=4)
    df_out['self_consumption_monthly'] = df.groupby(df.index.month)['Load'].apply(self_consumption_for_rolling, df)  # apply(self_sufficiency_for_rolling)  # pd.Timedelta(hours=4)

    df_out['Storage_Utilization'] = (df['BES'].abs()/250_000*100).rolling(pd.Timedelta(days=14)).mean()
    outputs[output_dir.name] = df_out
    

    name1, name2 = output_dir.name.split('_and_')
    if name1 not in self_consumption:
        outputs_table[name1]    = {}
        self_consumption[name1] = {}
        self_sufficiency[name1] = {}
        rmse_bes[name1]         = {}
        mae_bes[name1]          = {}
        sc_no_flex[name1]       = {}
        ss_no_flex[name1]       = {}

    outputs_table[name1][name2]    = df_out
    self_consumption[name1][name2] = calc_self_consumption(df['PV'].sum(axis=0), df.loc[df['Grid'] > 0, 'Grid'].sum(axis=0))
    self_sufficiency[name1][name2] = calc_self_sufficiency(df['Load'].sum(axis=0), df.loc[df['Grid'] < 0, 'Grid'].sum(axis=0))
    rmse_bes[name1][name2]         = np.sqrt(((output_bes[('StorageModel_0','power')] - output_bes[('StorageModel_0', 'actual_power')])**2).mean())
    mae_bes[name1][name2]          = (output_bes[('StorageModel_0','power')] - output_bes[('StorageModel_0', 'actual_power')]).abs().mean()

    res = df['PV']+df['Load']
    sc_no_flex[name1][name2]       = calc_self_consumption(df['PV'].sum(axis=0), res.loc[res > 0].sum(axis=0))
    ss_no_flex[name1][name2]       = calc_self_sufficiency(df['Load'].sum(axis=0), res.loc[res < 0].sum(axis=0))


    

In [3]:
fig, axs = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(width, height*0.5))

linestyle = ["solid", (0, (5, 10)), (0, (1, 10)), (0, (3, 10, 1, 10))]
cmap = mpl.colormaps['grey']
# cmap = mpl.colormaps['plasma']
colors = cmap(np.linspace(0, 1, 4))
ls_lookup = {}
color_lookup = {}

# sort labels, so they appear correctly sorted when plottin in the table legend
extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)

storage_model_names = [ind.replace('StorageModel', '').replace('SelfdischargeReducedMaxCap', 'Reduced max. cap, self-discharge').replace('ReducedMaxCap', 'Reduced max. capacity') for ind in outs.index]

lables = ['$\\downarrow$ CBES Models \\ Prediction $\\rightarrow$'] + storage_model_names 
handles = [extra] *(len(lables))

for name1, row in outs.items():
    lables.append(name1.replace('Prediction', '').replace('1', '').replace('LSTMLoadPerfectPv', 'LSTM load, perfect PV'))
    handles.append(extra)
    for name2, data in row.items():
        if name1 not in color_lookup:
            color_lookup[name1] = colors[len(color_lookup)]
        if name2 not in ls_lookup:
            ls_lookup[name2] = linestyle[len(ls_lookup)]
            
        c=color_lookup[name1]
        ls=ls_lookup[name2]
        
        name = name1 + ' ' + name2
        axs[0].plot(data.index, data['self_sufficiency_rolling'], color=c, linestyle=ls, label=name)

        h = axs[1].plot(data.index, data['self_consumption_rolling'], color=c, linestyle=ls, label=name) # self_consumption_monthly

        # axs[2].plot(data.index, data['Storage_Utilization'], color=c, linestyle=ls) # , label=name)
        # axs[2].set_ylabel('Storage-Utilization, %')  # Storage-Utilization
        # l = f'{name1}, {name2}'

        # h, l = axs[2].get_legend_handles_labels()
        lables.append('')
        handles.append(h[0])

axs[0].set_ylabel('Self-sufficiency, %') # Self Sufficiency
axs[0].grid(linestyle=':', linewidth=0.4)

axs[1].grid(linestyle=':', linewidth=0.4)
axs[1].set_ylabel('Self-consumption, %')  # Self-Consumption
axs[1].set_xlim(pd.to_datetime('2010-01-01 01:00:00+01:00'), pd.to_datetime('2010-12-31 23:00:00+01:00'))


# locale.setlocale(locale.LC_ALL, 'de_DE')


locator = mdates.MonthLocator() #  AutoDateLocator(minticks=13)
formatter = mdates.ConciseDateFormatter(locator)
formatter.formats = ['%y',  # ticks are mostly years
                    '%b',       # ticks are mostly months
                    '%d',       # ticks are mostly days
                    '%H:%M',    # hrs
                    '%H:%M',    # min
                    '%S.%f', ]  # secs
# these are mostly just the level above...
# formatter.zero_formats = [''] + formatter.formats[:-1]
formatter.zero_formats = formatter.formats
# ...except for ticks that are mostly hours, then it is nice to have
# month-day:
# formatter.zero_formats[3] = '%d-%b'

formatter.offset_formats = ['','%Y','%Y','%b %Y','%b %Y', '%b %Y %H:%M']
# formatter.offset_formats = ['%Y','%Y','%Y','%Y','%Y', '%Y']

axs[1].xaxis.set_major_locator(locator)
axs[1].xaxis.set_major_formatter(formatter)
axs[1].set_xlabel('Month of Year 2010')



# legend_handles = np.concatenate((legend_handles[::3],legend_handles[1::3]),axis=0)
# legend_labels = np.concatenate((legend_labels[::3],legend_labels[1::3]),axis=0)

# fig.legend(legend_handles, legend_labels, bbox_to_anchor=(0.5, 1), loc='lower center', ncol=3)
lgd = fig.legend(handles, lables, bbox_to_anchor=(0.5, 1), loc='lower center', ncol=4, handletextpad = -2.6)


fig.tight_layout()
fig.savefig('../../figures/enova/yearly_ss_and_sc.svg', dpi=300, bbox_extra_artists=(lgd, ), bbox_inches='tight')


NameError: name 'plt' is not defined